# Improving your dance application

In [12]:
import numpy as np
import cv2
import time
import mediapipe as mp
import datetime
import statistics as stat
from ffpyplayer.player import MediaPlayer

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [13]:
# Define function Angle Difference

def calculate_angle(first,mid,end):
    
    a = np.array(first) # First
    b = np.array(mid) # Mid
    c = np.array(end) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [14]:
# Define function Average Value

def average_value(value_list):
    
    average = sum(value_list)/len(value_list)
    
    return average

In [18]:
# Define function for Cosine Similarity Distance

def calculate_cosinesim(source, test):
    
    a = np.matmul(np.transpose(source), test)
    b = np.sum(np.multiply(source, source))
    c = np.sum(np.multiply(test, test))
    
    return (a / (np.sqrt(b)*np.sqrt(c)))

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

## Improve Your Korea Dance (K-pop Dance Comparison)

In [19]:
# Self-Camera
cap = cv2.VideoCapture(0)

## Inputing number for choose the song of dance.
song = input("Which song do you want to dance? (1 or 2 or 3): ")

while song not in ('1','2','3'):
    
    print('Wrong input!')
    song = input("Which dance do you want to dance? (1 or 2 or 3): ")
    
if song == '1':
    bgv = cv2.VideoCapture("./Video_Dataset/Psycho_original.mp4") 
    video_path= "./Video_Dataset/Psycho_original.mp4"

if song == '2': 
    bgv = cv2.VideoCapture("./Video_Dataset/LoveDive_original.mp4")
    video_path= "./Video_Dataset/LoveDive_original.mp4"
    
if song == '3':
    bgv = cv2.VideoCapture("./Video_Dataset/AnySong_original.mp4") 
    video_path= "./Video_Dataset/AnySong_original.mp4"

## Inputing number how accurate for improving your dance.
accurate_dance = input("How accurate for dance level? (Excellent(1) or Good(2) or NotBad(3)): ")

while accurate_dance not in ('1','2','3'):
    
    print('Wrong input!')
    accurate_dance = input("How accurate for dance level? (1 or 2 or 3): ")
    
if accurate_dance == '1': 
    acc_dance = 7

if accurate_dance == '2': 
    acc_dance = 5
    
if accurate_dance == '3': 
    acc_dance = 3

## Global Parameter for empty parameters
compare_collection = []
compare_collection_sim = []
s_nose1 = []
s_elbow_L1 = []
s_elbow_R1 = []
s_shoulder_L1 = []
s_shoulder_R1 = []
s_hip_L1 = []
s_hip_R1 = []
s_knee_L1 = []
s_knee_R1 = []
diff_n_all = []
diff_e_L_all = []
diff_e_R_all = []
diff_s_L_all = []
diff_s_R_all = []
diff_h_L_all = []
diff_h_R_all = []   
diff_k_L_all = []
diff_k_R_all = []
list_percent = []
n = 1
frame_array = []

## Q/A asking for recording Video
mode = input("Do you want to record the result as a video? (Y/N): ").upper()

while mode not in ('Y', 'N'):
    
    print('Wrong input!')
    mode = input("Do you want to record the result as a video? (Y/N): ").upper()
    
## Setup time sleep for build waited time a bit (3 seconds)
time.sleep(3)

## Counting Time
x = datetime.datetime.now().replace(microsecond=0)

## Prepaing opening song's sound
player = MediaPlayer(video_path)

## Setup mediapipe 
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    while True:
        
        ret1, bg1 = bgv.read()
        ret2, hm2 = cap.read()
        audio_frame, val = player.get_frame()
        prevTime = time.time()

        if bg1 is None:
            break 
            
        if hm2 is None:
            break 
            
        rt = 0.7            
        whm = int(bg1.shape[1] *rt*1.4)
        hhm = int(hm2.shape[0] *rt*1.4)
    
        frame1 = cv2.resize(bg1, (whm, hhm))
        frame2 = cv2.resize(hm2, (whm, hhm))

        # Recolor image to RGB
        image1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
        image2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
        image1.flags.writeable = False
        image2.flags.writeable = False

        # Make detection
        results1 = pose.process(image1)
        results2 = pose.process(image2)

        # Recolor back to BGR
        image1.flags.writeable = True
        image2.flags.writeable = True
        image1 = cv2.cvtColor(image1, cv2.COLOR_RGB2BGR)
        image2 = cv2.cvtColor(image2, cv2.COLOR_RGB2BGR)
    
        worst_position = 0
        
        # Extract landmarks
        try:
            landmarks1 = results1.pose_world_landmarks.landmark
            landmarks1_ = results1.pose_landmarks.landmark
            
            # Get coordinates
            nose1 = [landmarks1[mp_pose.PoseLandmark.NOSE.value].x,landmarks1[mp_pose.PoseLandmark.NOSE.value].y]
            wrist_L1 = [landmarks1[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks1[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            wrist_R1 = [landmarks1[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks1[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            elbow_L1 = [landmarks1[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks1[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            elbow_R1 = [landmarks1[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks1[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            shoulder_L1 = [landmarks1[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks1[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            shoulder_R1 = [landmarks1[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks1[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            hip_L1 = [landmarks1[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks1[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            hip_R1 = [landmarks1[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks1[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            knee_L1 = [landmarks1[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks1[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            knee_R1 = [landmarks1[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks1[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            ankle_L1 = [landmarks1[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks1[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            ankle_R1 = [landmarks1[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks1[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]   


            # Calculate angle
            angle_nose1 = round(calculate_angle(shoulder_L1, nose1, shoulder_R1),2)
            angle_elbow_L1 = round(calculate_angle(shoulder_L1, elbow_L1, wrist_L1),2)
            angle_elbow_R1 = round(calculate_angle(shoulder_R1, elbow_R1, wrist_R1),2)
            angle_shoulder_L1 = round(calculate_angle(elbow_L1, shoulder_L1, hip_L1),2)
            angle_shoulder_R1 = round(calculate_angle(elbow_R1, shoulder_R1, hip_R1),2)
            angle_hip_L1 = round(calculate_angle(shoulder_L1, hip_L1, knee_L1),2)
            angle_hip_R1 = round(calculate_angle(shoulder_R1, hip_R1, knee_R1),2)            
            angle_knee_L1 = round(calculate_angle(ankle_L1, knee_L1, hip_L1),2)
            angle_knee_R1 = round(calculate_angle(ankle_R1, knee_R1, hip_R1),2)                       
            
            s_nose1.append(angle_nose1)
            s_elbow_L1.append(angle_elbow_L1)
            s_elbow_R1.append(angle_elbow_R1)
            s_shoulder_L1.append(angle_shoulder_L1)
            s_shoulder_R1.append(angle_shoulder_R1)
            s_hip_L1.append(angle_hip_L1)
            s_hip_R1.append(angle_hip_R1)
            s_knee_L1.append(angle_knee_L1)
            s_knee_R1.append(angle_knee_R1)      

            landmarks2 = results2.pose_world_landmarks.landmark            
            landmarks2_ = results2.pose_landmarks.landmark
           
            # Get coordinates
            nose2 = [landmarks2[mp_pose.PoseLandmark.NOSE.value].x,landmarks2[mp_pose.PoseLandmark.NOSE.value].y]
            wrist_L2 = [landmarks2[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks2[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            wrist_R2 = [landmarks2[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks2[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            elbow_L2 = [landmarks2[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks2[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            elbow_R2 = [landmarks2[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks2[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            shoulder_L2 = [landmarks2[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks2[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            shoulder_R2 = [landmarks2[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks2[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            hip_L2 = [landmarks2[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks2[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            hip_R2 = [landmarks2[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks2[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            knee_L2 = [landmarks2[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks2[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            knee_R2 = [landmarks2[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks2[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            ankle_L2 = [landmarks2[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks2[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            ankle_R2 = [landmarks2[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks2[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            # Calculate angle
            angle_nose2 = round(calculate_angle(shoulder_L2, nose2, shoulder_R2),2)
            angle_elbow_L2 = round(calculate_angle(shoulder_L2, elbow_L2, wrist_L2),2)
            angle_elbow_R2 = round(calculate_angle(shoulder_R2, elbow_R2, wrist_R2),2)
            angle_shoulder_L2 = round(calculate_angle(elbow_L2, shoulder_L2, hip_L2),2)
            angle_shoulder_R2 = round(calculate_angle(elbow_R2, shoulder_R2, hip_R2),2)
            angle_hip_L2 = round(calculate_angle(shoulder_L2, hip_L2, knee_L2),2)
            angle_hip_R2 = round(calculate_angle(shoulder_R2, hip_R2, knee_R2),2) 
            angle_knee_L2 = round(calculate_angle(ankle_L2, knee_L2, hip_L2),2)
            angle_knee_R2 = round(calculate_angle(ankle_R2, knee_R2, hip_R2),2)
            
            compare_nose = abs(round(angle_nose2-angle_nose1,2))
            compare_elbow_L = abs(round(angle_elbow_L2-angle_elbow_L1,2))
            compare_elbow_R = abs(round(angle_elbow_R2-angle_elbow_R1,2))
            compare_shoulder_L = abs(round(angle_shoulder_L2-angle_shoulder_L1,2)) 
            compare_shoulder_R = abs(round(angle_shoulder_R2-angle_shoulder_R1,2))            
            compare_hip_L = abs(round(angle_hip_L2-angle_hip_L1,2)) 
            compare_hip_R = abs(round(angle_hip_R2-angle_hip_R1,2))
            compare_knee_L = abs(round(angle_knee_L2-angle_knee_L1,2)) 
            compare_knee_R = abs(round(angle_knee_R2-angle_knee_R1,2)) 
            
            compare_collection.append(compare_nose)
            compare_collection.append(compare_elbow_L)
            compare_collection.append(compare_elbow_R)    
            compare_collection.append(compare_shoulder_L)
            compare_collection.append(compare_shoulder_R)
            compare_collection.append(compare_hip_L)
            compare_collection.append(compare_hip_R)
            compare_collection.append(compare_knee_L)
            compare_collection.append(compare_knee_R)
            
            compare_score = average_value(compare_collection)
            
            # Calculate CosineSimilarity
            cosinesim_nose = calculate_cosinesim(nose1,nose2)
            cosinesim_elbow_L = calculate_cosinesim(elbow_L1,elbow_L2)
            cosinesim_elbow_R = calculate_cosinesim(elbow_R1,elbow_R2)
            cosinesim_shoulder_L = calculate_cosinesim(shoulder_L1,shoulder_L2)
            cosinesim_shoulder_R = calculate_cosinesim(shoulder_R1,shoulder_R2)            
            cosinesim_hip_L = calculate_cosinesim(hip_L1,hip_L2)
            cosinesim_hip_R = calculate_cosinesim(hip_R1,hip_R2)
            cosinesim_knee_L = calculate_cosinesim(knee_L1,knee_L2)
            cosinesim_knee_R = calculate_cosinesim(knee_R1,knee_R2)
            
            compare_collection_sim.append(cosinesim_nose)
            compare_collection_sim.append(cosinesim_elbow_L)
            compare_collection_sim.append(cosinesim_elbow_R)    
            compare_collection_sim.append(cosinesim_shoulder_L)
            compare_collection_sim.append(cosinesim_shoulder_R)
            compare_collection_sim.append(cosinesim_hip_L)
            compare_collection_sim.append(cosinesim_hip_R)
            compare_collection_sim.append(cosinesim_knee_L)
            compare_collection_sim.append(cosinesim_knee_R)                       
            
            compare_score_sim = average_value(compare_collection_sim)
            
            #Diff
            diff_n = abs(angle_nose2 - angle_nose1)
            diff_e_L = abs(angle_elbow_L2 - angle_elbow_L1)
            diff_e_R = abs(angle_elbow_R2 - angle_elbow_R1)
            diff_s_L = abs(angle_shoulder_L2 - angle_shoulder_L1)
            diff_s_R = abs(angle_shoulder_R2 - angle_shoulder_R1)
            diff_h_L = abs(angle_hip_L2 - angle_hip_L1)
            diff_h_R = abs(angle_hip_R2 - angle_hip_R1)    
            diff_k_L = abs(angle_knee_L2 - angle_knee_L1)
            diff_k_R = abs(angle_knee_R2 - angle_knee_R1)
         
            diff_n_all.append(diff_n)
            diff_e_L_all.append(diff_e_L)
            diff_e_R_all.append(diff_e_R)
            diff_s_L_all.append(diff_s_L)
            diff_s_R_all.append(diff_s_R)
            diff_h_L_all.append(diff_h_L)
            diff_h_R_all.append(diff_h_R)
            diff_k_L_all.append(diff_k_L)
            diff_k_R_all.append(diff_k_R)

            # Timing
            y = datetime.datetime.now().replace(microsecond=0)
            z = y-x
                                              
            # Visualize angle
            cv2.putText(image1, str(angle_nose1), 
                            tuple(np.multiply(nose1, [100, 480]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                            )
            cv2.putText(image2, str(angle_nose2), 
                            tuple(np.multiply(nose2, [100, 480]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                            ) 
                                              
            # Visualize Scoreborad
            cv2.rectangle(image1, (0,0), (265,63), (245,117,16), -1)
            cv2.putText(image1, 'CUMULATIVE SIMILARITY', (10,15),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image1, 'SCORE', (200,15),  
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image1, 'Score', (120,50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
            cv2.putText(image1, str(z), (0,image1.shape[0]-5), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
        
            # Render detections
            mp_drawing.draw_landmarks(image1, results1.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=1), 
                                            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=1) 
                                                )

            mp_drawing.draw_landmarks(image2, results2.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=1), 
                                        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=1) 
                                         )
            #1
            if diff_n > 7.77 :  

                cv2.circle(image2,
                           (int((landmarks2_[mp_pose.PoseLandmark.NOSE.value].x)*(image2.shape[1])),
                            int((landmarks2_[mp_pose.PoseLandmark.NOSE.value].y)*(image2.shape[0])))
                           ,4,(0,0,255),-1)
                worst_position += 1
            #2
            if diff_e_L > 24.47 :

                cv2.circle(image2,
                           (int((landmarks2_[mp_pose.PoseLandmark.LEFT_ELBOW.value].x)*(image2.shape[1])),
                            int((landmarks2_[mp_pose.PoseLandmark.LEFT_ELBOW.value].y)*(image2.shape[0])))
                           ,5,(0,0,255),-1)
                worst_position += 1
                
            #3
            if diff_e_R > 26.15 :

                cv2.circle(image2,
                           (int((landmarks2_[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x)*(image2.shape[1])),
                            int((landmarks2_[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y)*(image2.shape[0])))
                           ,5,(0,0,255),-1)
                worst_position += 1

            #4
            if diff_s_R > 16.19 :  

                cv2.circle(image2,
                           (int((landmarks2_[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x)*(image2.shape[1])),
                            int((landmarks2_[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y)*(image2.shape[0])))
                           ,5,(0,0,255),-1)
                worst_position += 1
            #5
            if diff_s_L > 15.27 :  

                cv2.circle(image2,
                           (int((landmarks2_[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x)*(image2.shape[1])),
                            int((landmarks2_[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y)*(image2.shape[0])))
                           ,5,(0,0,255),-1)        
                worst_position += 1
            #6
            if diff_h_L > 5.42 :  

                cv2.circle(image2,
                           (int((landmarks2_[mp_pose.PoseLandmark.LEFT_HIP.value].x)*(image2.shape[1])),
                            int((landmarks2_[mp_pose.PoseLandmark.LEFT_HIP.value].y)*(image2.shape[0])))
                           ,5,(0,0,255),-1)  
                worst_position += 1
            #7
            if diff_h_R > 6.09 :  

                cv2.circle(image2,
                           (int((landmarks2_[mp_pose.PoseLandmark.RIGHT_HIP.value].x)*(image2.shape[1])),
                            int((landmarks2_[mp_pose.PoseLandmark.RIGHT_HIP.value].y)*(image2.shape[0])))
                           ,5,(0,0,255),-1)  
                worst_position += 1
            #8
            if diff_k_L > 9.32 :  

                cv2.circle(image2,
                           (int((landmarks2_[mp_pose.PoseLandmark.LEFT_KNEE.value].x)*(image2.shape[1])),
                            int((landmarks2_[mp_pose.PoseLandmark.LEFT_KNEE.value].y)*(image2.shape[0])))
                           ,5,(0,0,255),-1)
                worst_position += 1
            #9
            if diff_k_R > 10.73 :  

                cv2.circle(image2,
                           (int((landmarks2_[mp_pose.PoseLandmark.RIGHT_KNEE.value].x)*(image2.shape[1])),
                            int((landmarks2_[mp_pose.PoseLandmark.RIGHT_KNEE.value].y)*(image2.shape[0])))
                           ,5,(0,0,255),-1) 
                worst_position += 1
            
            # Cumulative Similarity Score
            percent_sim = ((9-worst_position)/9)*100
            list_percent.append(percent_sim)
            aver_percent = sum(list_percent)/len(list_percent)
            cv2.putText(image1, str(round(aver_percent,2)), 
                        (10,50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)        

            # to calculate FPS
            currTime = time.time()
            diff_sec = currTime - prevTime 
            fps = 1 / diff_sec
            prevTime = currTime
            flip_img = cv2.flip(image2,1)
            
            # Showing Frame per second
            cv2.putText(flip_img, f'FPS: {int(fps)}', (image2.shape[1]-80, image2.shape[0]-5), cv2.FONT_HERSHEY_PLAIN, 1, (0, 20, 255), 2)
            
            result = np.hstack((image1,flip_img))
            cv2.imshow("Compare", result)
            frame_array.append(result)
  
        except:
            pass        
        
        if worst_position >= acc_dance :
            cv2.imwrite('./Not_corrent'+str(n)+'.jpg',result)
            n += 1
        
        # Wait 1 millisecond for any key press        
        if (cv2.waitKey(1)== 27):       # press ESC to quit
            break
    
    # Recording Video
    if mode == 'Y':
        FPS = 9.0
        size = (result.shape[1],result.shape[0])
        video_output = 'improve_yourdance_record.mp4'
        out = cv2.VideoWriter(video_output,cv2.VideoWriter_fourcc(*'FMP4'), FPS, size)
        for i in range(len(frame_array)):
            # writing to a image array
            out.write(frame_array[i]) if mode == 'Y' else None
        out.release()
        
    # Concluding Summary Similarity Score                                         
    print(f'Total Summary Similarity Score : {round(aver_percent,2)}')
    
    cv2.destroyAllWindows() # close all windows
    player.close_player()
    if bgv.isOpened():
        bgv.release() 
    if cap.isOpened():
        cap.release()

Which song do you want to dance? (1 or 2 or 3): 1
How accurate for dance level? (Excellent(1) or Good(2) or NotBad(3)): 1
Do you want to record the result as a video? (Y/N): N
16.746666666666666
25.71722222222222
29.464814814814805
32.433611111111105
34.22199999999999
36.300370370370366
37.82349206349207
37.78041666666667
37.55456790123457
38.415222222222226
38.11525252525253
39.65268518518519
40.79196581196581
41.68333333333332
41.24955555555555
40.92152777777777
40.52830065359476
40.627222222222215
40.92087719298245
41.226222222222226
43.29047619047619
43.060101010101
43.017342995169074
43.3548148148148
43.56666666666667
43.132435897435904
42.880205761316866
42.764880952380935
42.54099616858236
42.11762962962962
41.688243727598554
41.44947916666665
41.09232323232322
40.549183006535934
Total Summary Similarity Score : 24.51
